<a href="https://colab.research.google.com/github/ekaterinatao/house_md_tg_bot/blob/main/generative/NLP_2_HW2_house_md_bot_tinyllama_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Код запускается коррекно только на `GPU`  
  
### Графики обучения [по ссылке](https://wandb.ai/taoea/house_md_bot_generative/reports/-TyniLlama--Vmlldzo3MTAzNzk0?accessToken=ox6xn1fmrt7ocxy9lzqp8wr50oqxbcb1ooucfdnj97vin1p52ah76vc5o8pw0xo9)  
### Чек-пойнт дообученной модели [по ссылке](https://huggingface.co/ekaterinatao/house-md-tynyLlama/tree/**main**)

In [ ]:
!pip install trl transformers accelerate git+https://github.com/huggingface/peft.git -Uqqq
!pip install bitsandbytes einops datasets wandb -Uqqq

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

In [ ]:
import os
import random
from tqdm.auto import tqdm, trange
from dataclasses import dataclass
import datasets
import numpy as np
import pandas as pd
import torch
from transformers import (AutoTokenizer, AutoModel,
                          AutoModelForCausalLM,
                          BitsAndBytesConfig,
                          TrainingArguments,
                          GenerationConfig)
from peft import get_peft_model, PeftConfig, PeftModel, LoraConfig, prepare_model_for_kbit_training
from trl import SFTTrainer
import wandb

In [ ]:
@dataclass
class TrainingConfig:
    seed = 64
    dataset = 'ekaterinatao/house_md_context3'
    checkpoint = 'PY007/TinyLlama-1.1B-step-50K-105b'
    batch_size = 8
    device = "cuda" if torch.cuda.is_available() else "cpu"
    learning_rate = 2e-3
    num_epochs = 1
    weight_decay = 0.001
    gradient_accumulation_steps = 2
    optim = "paged_adamw_32bit"
    max_grad_norm = 0.3
    max_steps = 1000
    warmup_ratio = 0.03
    lr_scheduler_type = "constant"
    lora_alpha = 32
    lora_dropout = 0.05
    lora_rank = 32

config = TrainingConfig()

In [ ]:
seed = config.seed

random.seed(seed)
np.random.seed(seed)

torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Датасет

In [ ]:
dataset = datasets.load_dataset(config.dataset, split='train')
dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/77505 [00:00<?, ? examples/s]

Dataset({
    features: ['answer', 'question', 'context', 'labels', 'joined'],
    num_rows: 77505
})

In [ ]:
# для финального обучения отфильтрованы ответы только др. Хауса
dataset = dataset.filter(lambda example: example['labels'] == 0)
len(dataset)

Filter:   0%|          | 0/77505 [00:00<?, ? examples/s]

65088

In [ ]:
def make_prompt(example):
    try:
        context = example['context'][0]
    except:
        context = example['context']

    prompt = f"[INST]"
    prompt += f'Use the given context to guide your about answering the question\n\n'
    prompt += f"question: {example['question'][0]}\n\n"
    prompt += f"context: {context}\n\n"
    prompt += f"answer:{example['answer']}[/INST]\n"
    example['prompt'] = prompt

    return example

In [ ]:
dataset = dataset.map(make_prompt)
dataset

Map:   0%|          | 0/65088 [00:00<?, ? examples/s]

Dataset({
    features: ['answer', 'question', 'context', 'labels', 'joined', 'prompt'],
    num_rows: 65088
})

In [ ]:
dataset[0]['prompt']

"[INST]Use the given context to guide your about answering the question\n\nquestion: Two surgeries for multiple fractures and burns.\n\ncontext: Call 911.\n\nanswer:I'm thinking the broken bones are a response to the building falling on her head.[/INST]\n"

# TinyLlama

In [ ]:
print(f'device is {config.device}')
if torch.cuda.is_available():
    print(torch.cuda.get_device_name())

device is cuda
Tesla T4


In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    config.checkpoint,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

config.json:   0%|          | 0.00/607 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.40G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/129 [00:00<?, ?B/s]

In [ ]:
llm_tokenizer = AutoTokenizer.from_pretrained(
    config.checkpoint,
    add_eos_token=True,
    trust_remote_code=True,
    padding_side='right'
)
llm_tokenizer.add_special_tokens({'pad_token': '[PAD]'})
llm_tokenizer.pad_token = llm_tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
model = prepare_model_for_kbit_training(model)

peft_config = LoraConfig(
    lora_alpha=config.lora_alpha,
    lora_dropout=config.lora_dropout,
    r=config.lora_rank,
    bias="none",
    task_type="CAUSAL_LM",
)

peft_model = get_peft_model(model, peft_config)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
output_dir = "/content/drive/MyDrive/models/tinyllama_2"

In [ ]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
os.environ["WANDB_PROJECT"]="house_md_bot_generative"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=config.num_epochs,
    per_device_train_batch_size=config.batch_size,
    gradient_accumulation_steps=config.gradient_accumulation_steps,
    optim=config.optim,
    save_steps=100,
    logging_steps=100,
    learning_rate=config.learning_rate,
    weight_decay=config.weight_decay,
    max_grad_norm=config.max_grad_norm,
    max_steps=config.max_steps,
    warmup_ratio=config.warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=config.lr_scheduler_type,
    report_to="wandb",
    run_name="only_house_1000",
)

In [ ]:
trainer = SFTTrainer(
    model=peft_model,
    train_dataset=dataset,
    peft_config=peft_config,
    max_seq_length=500,
    dataset_text_field='prompt',
    tokenizer=llm_tokenizer,
    args=training_arguments
)
peft_model.config.use_cache = False

Map:   0%|          | 0/65088 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

Step,Training Loss
100,1.871000
200,1.842600
300,1.822000
400,1.819400
500,1.807700
600,1.792300
700,1.793900
800,1.790100
900,1.781100
1000,1.759800


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.

TrainOutput(global_step=1000, training_loss=1.808001983642578, metrics={'train_runtime': 2918.4378, 'train_samples_per_second': 5.482, 'train_steps_per_second': 0.343, 'total_flos': 8242039250583552.0, 'train_loss': 1.808001983642578, 'epoch': 0.25})

In [ ]:
wandb.finish()

train/epoch,▁▂▃▃▄▅▆▆▇██
train/global_step,▁▂▃▃▄▅▆▆▇██
train/grad_norm,█▂▁▂▄▄▄▅▅▃
train/learning_rate,▁▁▁▁▁▁▁▁▁▁
train/loss,█▆▅▅▄▃▃▃▂▁
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁
train/train_samples_per_second,▁
train/train_steps_per_second,▁
train/epoch,0.25


# Inference

In [ ]:
dir = "/content/drive/MyDrive/models/tinyllama_2/checkpoint-1000"

In [ ]:
peft_config_eval = PeftConfig.from_pretrained(dir)

In [ ]:
model_id = "PY007/TinyLlama-1.1B-step-50K-105b"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

trained_model = AutoModelForCausalLM.from_pretrained(
    peft_config_eval.base_model_name_or_path,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    model_id,
    add_eos_token=True,
    trust_remote_code=True,
    padding_side='right'
)
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
def generate_answer(query, model):
    encoding = tokenizer(query, return_tensors="pt").to(config.device)
    generation_config = GenerationConfig(
        max_new_tokens=70,
        pad_token_id = tokenizer.eos_token_id,
        repetition_penalty=1.5,
        eos_token_id = tokenizer.eos_token_id
    )
    outputs = model.generate(
        input_ids=encoding.input_ids,
        generation_config=generation_config
    )
    text_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return text_output.split("\n")[-1]

def get_test_prompt(dataset):
    id = random.randint(0, len(dataset))
    question = dataset[id]['question'][0]
    answer = dataset[id]['answer']
    try:
        context = dataset[id]['context'][0]
    except:
        context = dataset[id]['context']
    prompt = f"[INST]"
    prompt += f'Use the given context to guide your about answering the question\n\n'
    prompt += f"question: {question}\n\n"
    prompt += f"context: {context}\n\n"
    prompt += f"answer:[/INST]\n"
    return question, answer, prompt

### Пример генерации
Генерация ответов на вопросы, которые присутствовали в датасете (`Generated answer`)  
Сгенерированные ответы близки по смыслу вопросу, возможно более интересные, чем оригинальный ответ.

In [ ]:
question, answer, prompt = get_test_prompt(dataset)
answer_gen = generate_answer(prompt, trained_model)

print(f"\nQuestion: {question}\n")
print(f"Generated answer: {answer_gen}\n")
print(f'Real answer in dataset: {answer}')

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.



Question: Ulcers secondary to vaculitis.

Generated answer: about: Create an issue for bugs and feature

Real answer in dataset: No, that's just sores, not obesity.


In [ ]:
question, answer, prompt = get_test_prompt(dataset)
answer_gen = generate_answer(prompt, trained_model)

print(f"\nQuestion: {question}\n")
print(f"Generated answer: {answer_gen}\n")
print(f'Real answer in dataset: {answer}')

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.



Question: There have been new anti-psychotics dEveloped since he ran away. He's been on them for a couple of days, and by tonight, he should be in shape to talk to me.

Generated answer: : The answer is that there are no drugs yet approved as antidepressants or even prescribed medications; however, many people do take these types of meds because their symptoms seem so similar to those associated with depression (easily treatable). It may also help to know what other diseases might cause this type of

Real answer in dataset: But you're not sure if he wants to.
